# A Python software engineer

Some insights. This is the beta verions:
- Sometimes the code interpreter cannot generate output files, it depends on the prompting.
- Sometimes, the source code generated have sintax errors.

In [1]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
import os

# en windows es necesario especificar la ruta del .env
load_dotenv(Path("C:\projects\rag\.env"))
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

This is an example of a Python software engineer who can refactor, develop test units and run test units.

In [2]:
file = client.files.create(
  file=open("play_tetris.py", "rb"),
  purpose='assistants'
)

Some times, we cannot downlod the outputfile. We must ensure to include in instructinos, the name of the file and "Make the file id available to download".

In [10]:
output_file_name = "play_tetris_refactored.py"
assistant_instruction = r"""Generate {} file, always. Make the file id available to download. You are software engineer, your goal is to help another engineer write code using Python. 
  Focus on writting clean, performant and concise code to solve the problem.  """.format(output_file_name)

assistant = client.beta.assistants.create(
  name="Code writer assistant",
  description="Code writer assistant",  
  model="gpt-4o",
  instructions = assistant_instruction,
  tools=[{"type": "code_interpreter"}],
  tool_resources={
    "code_interpreter": {
      "file_ids": [file.id]
    }
  }  
)

In [11]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Refactor this file.",
      "attachments": [
        {
          "file_id": file.id,
          "tools": [{"type": "code_interpreter"}]
        }
      ]
    }
  ]
)

In [12]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [13]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_DdCKOaY0pLE33AhNp5lydSxc', assistant_id='asst_3dXbn8iof9dgw4TCCCpVuJfy', attachments=[Attachment(file_id='file-ckY2zS5W45KnBmf5vQ2gsF7J', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=177, file_path=FilePath(file_id='file-ckY2zS5W45KnBmf5vQ2gsF7J'), start_index=134, text='sandbox:/mnt/data/play_tetris_refactored.py', type='file_path')], value='The refactored Tetris game script has been saved. You can download it using the following link:\n\n[Download play_tetris_refactored.py](sandbox:/mnt/data/play_tetris_refactored.py)\n\nFeel free to run and test the game. If you have any questions or need further modifications, let me know!'), type='text')], created_at=1725915699, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_4LeFeqL9NOUhvBl1Kz26YTpT', status=None, thread_id='thread

Parsing results

In [14]:
output = messages.data[0].content[0].text.value
print(messages.data[0].content[0].text)
print(messages.data[0].content[0].text.annotations)
print(output)

Text(annotations=[FilePathAnnotation(end_index=177, file_path=FilePath(file_id='file-ckY2zS5W45KnBmf5vQ2gsF7J'), start_index=134, text='sandbox:/mnt/data/play_tetris_refactored.py', type='file_path')], value='The refactored Tetris game script has been saved. You can download it using the following link:\n\n[Download play_tetris_refactored.py](sandbox:/mnt/data/play_tetris_refactored.py)\n\nFeel free to run and test the game. If you have any questions or need further modifications, let me know!')
[FilePathAnnotation(end_index=177, file_path=FilePath(file_id='file-ckY2zS5W45KnBmf5vQ2gsF7J'), start_index=134, text='sandbox:/mnt/data/play_tetris_refactored.py', type='file_path')]
The refactored Tetris game script has been saved. You can download it using the following link:

[Download play_tetris_refactored.py](sandbox:/mnt/data/play_tetris_refactored.py)

Feel free to run and test the game. If you have any questions or need further modifications, let me know!


In [15]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
file_path = messages.data[0].content[0].text.annotations[0].file_path.file_id
file_name = client.files.with_raw_response.retrieve_content(file_path)

with open(output_file_name, "wb") as file:  
    file.write(file_name.content) 

***Asking to see errors***

In [19]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Correct sintax errors")

In [20]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [21]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  output = messages.data[0].content[0].text.value
  print(output)
else:
  print(run.status)

The corrected Tetris game script has been saved. You can download it using the following link and run it locally to test:

[Download play_tetris_corrected.py](sandbox:/mnt/data/play_tetris_corrected.py)

Ensure you run it in an environment where interactive input is supported, such as a terminal or command prompt. If you encounter any issues, please let me know!


In [22]:
print(messages.data[0].content[0].text.annotations)

[]


Create a response asking for unit test

In [49]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Write a simple unit test for that file")

In [50]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [51]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_nVizZmGtlralgqpLFmA1duhd', assistant_id='asst_VMZqGRm8uMlfSR0Qjg7oTA6e', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='To write a simple unit test for the Tic-Tac-Toe game, we\'ll use Jest, a popular JavaScript testing framework. We\'ll test the core functions such as `updateCell`, `changePlayer`, `checkWinner`, and `restartGame`.\n\nEnsure that you have Jest installed in your project:\n```bash\nnpm install --save-dev jest\n```\n\nHere\'s how you can create a unit test file for the Tic-Tac-Toe game:\n\n```javascript\n// tic-tac-toe.test.js\n\n// Import the functions to test\nconst { updateCell, changePlayer, checkWinner, restartGame } = require(\'./tic-tac-toe\');\n\ndescribe(\'Tic-Tac-Toe Game\', () => {\n    let options, currentPlayer, running;\n\n    beforeEach(() => {\n        options = Array(9).fill("");\n        currentPlayer = "tomato";\n        running = true;\n    });\n\n    test(\'sh

In [55]:
import json

output = messages.data[0].content[0].text.value
print(output)

To write a simple unit test for the Tic-Tac-Toe game, we'll use Jest, a popular JavaScript testing framework. We'll test the core functions such as `updateCell`, `changePlayer`, `checkWinner`, and `restartGame`.

Ensure that you have Jest installed in your project:
```bash
npm install --save-dev jest
```

Here's how you can create a unit test file for the Tic-Tac-Toe game:

```javascript
// tic-tac-toe.test.js

// Import the functions to test
const { updateCell, changePlayer, checkWinner, restartGame } = require('./tic-tac-toe');

describe('Tic-Tac-Toe Game', () => {
    let options, currentPlayer, running;

    beforeEach(() => {
        options = Array(9).fill("");
        currentPlayer = "tomato";
        running = true;
    });

    test('should update cell correctly', () => {
        const cell = { textContent: "" };
        updateCell(cell, 0, options, currentPlayer);
        expect(options[0]).toBe("tomato");
        expect(cell.textContent).toBe("tomato");
    });

    test('sh

Asking for running the unit test. 

In [56]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Run the unit test and show the results")

In [58]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id)

In [59]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_yc0Hge1JvkLAmmQRuanN1mI0', assistant_id='asst_VMZqGRm8uMlfSR0Qjg7oTA6e', attachments=[Attachment(file_id='file-3HmNja5XeXFEyexp77G0ZGSd', tools=[CodeInterpreterTool(type='code_interpreter')]), Attachment(file_id='file-Vj2fQEzx95KG5ozUur0I75kl', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=188, file_path=FilePath(file_id='file-Vj2fQEzx95KG5ozUur0I75kl'), start_index=156, text='sandbox:/mnt/data/tic-tac-toe.js', type='file_path'), FilePathAnnotation(end_index=263, file_path=FilePath(file_id='file-3HmNja5XeXFEyexp77G0ZGSd'), start_index=226, text='sandbox:/mnt/data/tic-tac-toe.test.js', type='file_path')], value="The refactored JavaScript file and test file have been saved. You can download them using the links below:\n\n- [Refactored JavaScript file (tic-tac-toe.js)](sandbox:/mnt/data/tic-tac-toe.js)\n- [Test file (tic-tac-toe.test.js)]

In [60]:
import json

output = messages.data[0].content[0].text.value
print(output)

The refactored JavaScript file and test file have been saved. You can download them using the links below:

- [Refactored JavaScript file (tic-tac-toe.js)](sandbox:/mnt/data/tic-tac-toe.js)
- [Test file (tic-tac-toe.test.js)](sandbox:/mnt/data/tic-tac-toe.test.js)

Currently, I don't have the capability to run JavaScript code or Jest tests directly. You can run the test locally by following these steps:

1. Ensure you have Node.js and npm installed.
2. Install Jest using the command:
    ```bash
    npm install --save-dev jest
    ```
3. Run Jest tests using the command:
    ```bash
    npx jest tic-tac-toe.test.js
    ```

This will execute the test cases and provide you with the test results in your terminal.
